In [1]:
import os
import gc
import copy

import torch
from torch import nn
import pandas as pd

from glob import glob
from tqdm import tqdm
import numpy as np

from config import config
from dataset import FackFaceDetLoader
from model import FackFaceDetModel

{'checkpoint_config': <Box: {'interval': 1}>,
 'debug': False,
 'earlystopping_config': {'mode': 'max',
                          'monitor': 'f1_score',
                          'patience': 2},
 'epochs': 5,
 'image_size': 384,
 'image_size_tta': 440,
 'log_average_filter': <BoxList: []>,
 'log_config': {'hooks': <BoxList: [{'name': 'FackFaceDetLoggerHook'}]>,
                'interval': 50},
 'log_level': 'INFO',
 'lr_config': <Box: {'policy': 'OneCycle', 'max_lr': 2e-05, 'by_epoch': False}>,
 'model': <Box: {'name': 'swin_large_patch4_window12_384', 'output_dim': 1}>,
 'momentum_config': <Box: {'policy': 'OneCycle', 'by_epoch': False}>,
 'n_splits': 10,
 'name': 'FackFaceDet',
 'optimizer': <Box: {'name': 'AdamW'}>,
 'optimizer_config': <Box: {'grad_clip': None}>,
 'root': '../datasets/fackface_det/',
 'seed': 999,
 'train_loader': {'batch_size': 16,
                  'drop_last': True,
                  'num_workers': 10,
                  'pin_memory': False,
                  'sh

In [2]:
df = pd.read_csv(os.path.join(config.root, 'submission.csv'), sep='\t') if not config.debug else pd.read_csv(os.path.join(config.root, 'submission.csv'), sep='\t')[:100]
df_org = copy.deepcopy(df)
df['fnames'] = df['fnames'].apply(lambda x: os.path.join(config.root, 'image', 'test', x))
df.head()

,fnames,label
0,../datasets/fackface_det/image/test/test_0.jpg,1
1,../datasets/fackface_det/image/test/test_1.jpg,0
2,../datasets/fackface_det/image/test/test_2.jpg,1
3,../datasets/fackface_det/image/test/test_3.jpg,0
4,../datasets/fackface_det/image/test/test_4.jpg,1


In [3]:
import ttach as tta

test_loader = FackFaceDetLoader(df, df, config).test_dataloader()
models_dir = './checkpoint/swin_large_patch4_window12_384/commit/'
device = "cuda:1" 

predicted_labels = None
for model_name in glob(models_dir + '/*'):

    model = FackFaceDetModel(config)
    model.load_state_dict(torch.load(model_name)['state_dict'])
    model = model.cuda()
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(
        model, 
        tta.aliases.five_crop_transform(config.image_size,config.image_size)
    )
    
    temp_preds = None
    with torch.no_grad():
        for (images, target) in tqdm(test_loader, desc=f'Predicting. '):
            images = images.to("cuda", non_blocking=True)
            predictions = torch.sigmoid(tta_model(images)).to('cpu').numpy()
            
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))

    if predicted_labels is None:
        predicted_labels = temp_preds
    else:
        predicted_labels += temp_preds
    
#     del model
    gc.collect()
predicted_labels /= (len(glob(models_dir + '/*')))
predicted_labels[predicted_labels >= 0.5] = 1
predicted_labels[predicted_labels < 0.5] = 0

sub_df = pd.DataFrame()
sub_df['fnames'] = df_org['fnames']
sub_df['label'] = predicted_labels.astype(int)

sub_df.to_csv('submission.csv', index=False, sep='\t')

Predicting. : 100%|██████████| 505/505 [09:39<00:00,  1.15s/it]
